# Data processing

In [ ]:
!unzip /content/drive/MyDrive/data.zip 

Archive:  /content/drive/MyDrive/data.zip
   creating: data/
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/events.json        
  inflating: __MACOSX/data/._events.json  
  inflating: data/meta.json          
  inflating: __MACOSX/data/._meta.json  


In [ ]:
import json
# read meta data
with open('data/meta.json', 'r') as fp:
    products = json.load(fp)

In [ ]:
import pandas as pd

meta_product_ids = []
meta_product_name = []
meta_product_subcategory = []
meta_product_category = []
for product in products['meta']:
    meta_product_ids.append(product['productid'])
    meta_product_name.append(product['name'])
    meta_product_subcategory.append(product['subcategory'])
    meta_product_category.append(product['category'])

del products
products = pd.DataFrame()
products['product_id'] = meta_product_ids
products['name'] = meta_product_name
products['subcategory'] = meta_product_subcategory
products['category'] = meta_product_category
del meta_product_ids, meta_product_name, meta_product_subcategory, meta_product_category
products

,product_id,name,subcategory,category
0,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,Saç Bakımı,Kişisel Bakım
1,HBV00000BSAQG,Best Pet Jöle İçinde Parça Etli Somonlu Konser...,Kedi,Pet Shop
2,HBV00000JUHBA,Türkiye Tarım Kredi Koop.Yeşil Mercimek 1 kg,"Bakliyat, Pirinç, Makarna",Temel Gıda
3,HBV00000NE0QI,Namet Fıstıklı Macar Salam 100 gr,Şarküteri,"Et, Balık, Şarküteri"
4,HBV00000NE0UQ,Muratbey Burgu Peyniri 250 gr,Peynir,Kahvaltılık ve Süt
...,...,...,...,...
10231,ZYHPREISBBKL008,Reis Gönen Baldo Pirinç 1 kg,"Bakliyat, Pirinç, Makarna",Temel Gıda
10232,ZYNES11470137,Nescafé Classic Çözünebilir Kahve 200gr Ekopaket,Kahve,İçecekler
10233,ZYPAREX1909309,Parex Powerball Çizmez Ovma Teli,Ev Temizlik Ürünleri,Ev Bakım ve Temizlik
10234,ZYPAREX2107986,Parex Tornado Döner Başlıklı Temizlik Seti,Yüzey Temizleyiciler,Ev Bakım ve Temizlik


In [ ]:
# check if there is any null value in prodd
products.isnull().values.any()

True

In [ ]:
# delete null values
products = products.dropna()
products = products.reset_index()
del products['index']
products

,product_id,name,subcategory,category
0,HBV00000AX6LR,Palette Kalıcı Doğal Renkler 10-4 PAPATYA,Saç Bakımı,Kişisel Bakım
1,HBV00000BSAQG,Best Pet Jöle İçinde Parça Etli Somonlu Konser...,Kedi,Pet Shop
2,HBV00000JUHBA,Türkiye Tarım Kredi Koop.Yeşil Mercimek 1 kg,"Bakliyat, Pirinç, Makarna",Temel Gıda
3,HBV00000NE0QI,Namet Fıstıklı Macar Salam 100 gr,Şarküteri,"Et, Balık, Şarküteri"
4,HBV00000NE0UQ,Muratbey Burgu Peyniri 250 gr,Peynir,Kahvaltılık ve Süt
...,...,...,...,...
10230,ZYHPREISBBKL008,Reis Gönen Baldo Pirinç 1 kg,"Bakliyat, Pirinç, Makarna",Temel Gıda
10231,ZYNES11470137,Nescafé Classic Çözünebilir Kahve 200gr Ekopaket,Kahve,İçecekler
10232,ZYPAREX1909309,Parex Powerball Çizmez Ovma Teli,Ev Temizlik Ürünleri,Ev Bakım ve Temizlik
10233,ZYPAREX2107986,Parex Tornado Döner Başlıklı Temizlik Seti,Yüzey Temizleyiciler,Ev Bakım ve Temizlik


In [ ]:
# compressing name and subcategory for have little bit more features
def compress_features(data):
    features = []
    for i in range(0, data.shape[0]):
        datum = data['subcategory'][i] + ', ' + data['name'][i]
        features.append(datum)
    return features

In [ ]:
products['features'] = compress_features(products)
del products['subcategory'], products['name']
products

,product_id,category,features
0,HBV00000AX6LR,Kişisel Bakım,"Saç Bakımı, Palette Kalıcı Doğal Renkler 10-4 ..."
1,HBV00000BSAQG,Pet Shop,"Kedi, Best Pet Jöle İçinde Parça Etli Somonlu ..."
2,HBV00000JUHBA,Temel Gıda,"Bakliyat, Pirinç, Makarna, Türkiye Tarım Kredi..."
3,HBV00000NE0QI,"Et, Balık, Şarküteri","Şarküteri, Namet Fıstıklı Macar Salam 100 gr"
4,HBV00000NE0UQ,Kahvaltılık ve Süt,"Peynir, Muratbey Burgu Peyniri 250 gr"
...,...,...,...
10230,ZYHPREISBBKL008,Temel Gıda,"Bakliyat, Pirinç, Makarna, Reis Gönen Baldo Pi..."
10231,ZYNES11470137,İçecekler,"Kahve, Nescafé Classic Çözünebilir Kahve 200gr..."
10232,ZYPAREX1909309,Ev Bakım ve Temizlik,"Ev Temizlik Ürünleri, Parex Powerball Çizmez O..."
10233,ZYPAREX2107986,Ev Bakım ve Temizlik,"Yüzey Temizleyiciler, Parex Tornado Döner Başl..."


In [ ]:
import string

def clean_text(text): 
    black_list = ['ml', 'gr', 'lt', 'kg']
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>1))]) 
    text2 = text2.split(' ')
    text2 = ' '.join([word for word in text2 if (word not in black_list)])
    #print(text2)
    return text2.lower()

products['clean_features'] = products['features'].apply(clean_text)

In [ ]:
max_len = 0
for name in products['clean_features']:
    if len(name)>max_len:
        max_len = len(name)
        name2 = name
print('max lenght is ', max_len)
print('The string is ', name2)


max lenght is  131
The string is  çamaşır yıkama omo sıvı çamaşır deterjanı active beyazlar ve renkliler i̇çin en zorlu lekeleri i̇lk yıkamada çıkarır ml yıkama adet


In [ ]:
print('Number of category \t', products.category.nunique())
print('Numver of product \t', products.product_id.nunique())

Number of category 	 20
Numver of product 	 10235


In [ ]:
products.to_pickle('/content/drive/MyDrive/products')

# Model training

In [ ]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# init callback class
class my_callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
train_data = products.clean_features.to_list()
for i, data in enumerate(train_data):
    data = data.split(' ')
    train_data[i] = data

In [ ]:
import time
# init word2vec class
w2v_model = Word2Vec(sentences=train_data, size=300, window=5, min_count=1, workers=4, sample=1e-5, sg=1)

# build vocab
w2v_model.build_vocab(train_data, update=True)

# train the w2v model
start = time.time()
w2v_model.train(train_data, 
                total_examples=w2v_model.corpus_count, 
                epochs=10001, 
                report_delay=1,
                compute_loss = True, 
                callbacks=[my_callback()]) # add the callback class
end = time.time()

print("elapsedtime in seconds :"+ str(end - start))
# save the word2vec model
w2v_model.save('/content/drive/MyDrive/hepsi_burada_word2vec.model')

Loss after epoch 0: 35873.140625
Loss after epoch 100: 8120.375
Loss after epoch 200: 2933.75
Loss after epoch 300: 1658.0
Loss after epoch 400: 1952.5
Loss after epoch 500: 2229.5
Loss after epoch 600: 1643.25
Loss after epoch 700: 2172.25
Loss after epoch 800: 1430.75
Loss after epoch 900: 1638.5
Loss after epoch 1000: 1528.0
Loss after epoch 1100: 1581.75
Loss after epoch 1200: 1198.5
Loss after epoch 1300: 2064.0
Loss after epoch 1400: 1431.0
Loss after epoch 1500: 1596.5
Loss after epoch 1600: 1511.0
Loss after epoch 1700: 2197.0
Loss after epoch 1800: 2266.0
Loss after epoch 1900: 1791.0
Loss after epoch 2000: 1559.0
Loss after epoch 2100: 1499.0
Loss after epoch 2200: 1038.5
Loss after epoch 2300: 1576.5
Loss after epoch 2400: 1394.0
Loss after epoch 2500: 1414.5
Loss after epoch 2600: 1418.0
Loss after epoch 2700: 1180.0
Loss after epoch 2800: 1495.5
Loss after epoch 2900: 1403.0
Loss after epoch 3000: 1509.5
Loss after epoch 3100: 1498.5
Loss after epoch 3200: 2221.5
Loss afte

In [ ]:
w2v_model.callbacks = ()
w2v_model.save('/content/drive/MyDrive/den_hepsi_burada_word2vec.model')

# Load and try the model

In [1]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 132 kB 5.0 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219284 sha256=4399f7c1955762e460358f82fb0fc032916f61fafb8bf405164d8c6488836581
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [6]:
#  load data
def load_data(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

products = load_data('/content/drive/MyDrive/products')

In [11]:
from gensim.models import Word2Vec # you can delete this later. That code duplicated 
model = Word2Vec.load('/content/drive/MyDrive/den_hepsi_burada_word2vec.model')    

In [39]:
products.head()

,product_id,category,features,clean_features
0,HBV00000AX6LR,Kişisel Bakım,"Saç Bakımı, Palette Kalıcı Doğal Renkler 10-4 ...",saç bakımı palette kalıcı doğal renkler papatya
1,HBV00000BSAQG,Pet Shop,"Kedi, Best Pet Jöle İçinde Parça Etli Somonlu ...",kedi best pet jöle i̇çinde parça etli somonlu ...
2,HBV00000JUHBA,Temel Gıda,"Bakliyat, Pirinç, Makarna, Türkiye Tarım Kredi...",bakliyat pirinç makarna türkiye tarım kredi ko...
3,HBV00000NE0QI,"Et, Balık, Şarküteri","Şarküteri, Namet Fıstıklı Macar Salam 100 gr",şarküteri namet fıstıklı macar salam
4,HBV00000NE0UQ,Kahvaltılık ve Süt,"Peynir, Muratbey Burgu Peyniri 250 gr",peynir muratbey burgu peyniri


In [12]:
words = list(model.wv.vocab)
print('Vocab size: ' + str(len(words)))

Vocab size: 6110


In [16]:
import string

def clean_text(text):
    clean_list = []
    for item in text:
        black_list = ['ml', 'gr', 'lt', 'kg']
        delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
        delete_dict[' '] = ' ' 
        table = str.maketrans(delete_dict)
        text1 = item.translate(table)
        textArr= text1.split()
        text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>1))]) 
        text2 = text2.split(' ')
        text2 = ' '.join([word for word in text2 if (word not in black_list)])
        clean_list.append(text2.lower())
    return clean_list

In [21]:
products.features[10232]

'Ev Temizlik Ürünleri, Parex Powerball Çizmez Ovma Teli'

In [88]:
cart = ['şarküteri namet fıstıklı macar salam', 'peynir muratbey burgu peyniri']

In [89]:
clean_cart = clean_text(cart)

In [90]:
import numpy as np

def get_cart_avg(data):
    '''
    data: list of text
    '''
    cart_vectors = np.zeros((300,))
    c = 0 
    for sent in data:
        sent = sent.split(' ')
        for word in sent:
            word_vector = model.wv.get_vector(word)
            cart_vectors += word_vector
            c += 1
    return cart_vectors / c

In [91]:
avg_cart = get_cart_avg(clean_cart)
avg_cart.shape

(300,)

In [92]:
avg_products = []
for i in range(products.shape[0]):
    avg_product = get_cart_avg([products.clean_features[i]])
    avg_products.append(avg_product)
products['avg'] = avg_products
products

,product_id,category,features,clean_features,avg
0,HBV00000AX6LR,Kişisel Bakım,"Saç Bakımı, Palette Kalıcı Doğal Renkler 10-4 ...",saç bakımı palette kalıcı doğal renkler papatya,"[-0.10433321107744373, -0.40376642772129606, 0..."
1,HBV00000BSAQG,Pet Shop,"Kedi, Best Pet Jöle İçinde Parça Etli Somonlu ...",kedi best pet jöle i̇çinde parça etli somonlu ...,"[-0.5698804662873348, -0.02613970699409644, 0...."
2,HBV00000JUHBA,Temel Gıda,"Bakliyat, Pirinç, Makarna, Türkiye Tarım Kredi...",bakliyat pirinç makarna türkiye tarım kredi ko...,"[-0.3286075004272991, -0.37011471385550165, -0..."
3,HBV00000NE0QI,"Et, Balık, Şarküteri","Şarküteri, Namet Fıstıklı Macar Salam 100 gr",şarküteri namet fıstıklı macar salam,"[0.11705920174717903, -0.36727973222732546, 0...."
4,HBV00000NE0UQ,Kahvaltılık ve Süt,"Peynir, Muratbey Burgu Peyniri 250 gr",peynir muratbey burgu peyniri,"[-0.6765416376292706, -0.31077154632657766, -0..."
...,...,...,...,...,...
10230,ZYHPREISBBKL008,Temel Gıda,"Bakliyat, Pirinç, Makarna, Reis Gönen Baldo Pi...",bakliyat pirinç makarna reis gönen baldo pirinç,"[-0.23651108784335, -0.2668937166620578, -0.06..."
10231,ZYNES11470137,İçecekler,"Kahve, Nescafé Classic Çözünebilir Kahve 200gr...",kahve nescafé classic çözünebilir kahve 200gr ...,"[-0.2725622398512704, -0.19013077127082006, 0...."
10232,ZYPAREX1909309,Ev Bakım ve Temizlik,"Ev Temizlik Ürünleri, Parex Powerball Çizmez O...",ev temizlik ürünleri parex powerball çizmez ov...,"[-0.08983673580223694, -0.33764087967574596, 0..."
10233,ZYPAREX2107986,Ev Bakım ve Temizlik,"Yüzey Temizleyiciler, Parex Tornado Döner Başl...",yüzey temizleyiciler parex tornado döner başlı...,"[-0.3365426016971469, -0.3512522731907666, 0.2..."


In [93]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(products.avg[10233].reshape(1,-1), products.avg[10232].reshape(1,-1))

array([[0.71324077]])

In [98]:
similar = []
for i in range(len(products)):
    similarity = cosine_similarity(avg_cart.reshape(1,-1), products.avg[i].reshape(1,-1))
    if similarity > 0.8:
        item = {
            'productName':products.features[i],
            'similarity':similarity
        }
        similar.append(item)

In [108]:
recommended = sorted(similar, key = lambda i: i['similarity'], reverse=True)
recommended[:10] 

[{'productName': 'Şarküteri, Namet Fıstıklı Macar Salam 100 gr',
  'similarity': array([[0.88740049]])},
 {'productName': 'Şarküteri, Namet Macar Salam 100 gr',
  'similarity': array([[0.8848124]])},
 {'productName': 'Şarküteri, Namet Dilimli Fıstıklı Salam 150 gr',
  'similarity': array([[0.88009244]])},
 {'productName': 'Şarküteri, Namet Dilimli Dana Macar Salam 200 gr',
  'similarity': array([[0.87193235]])},
 {'productName': 'Şarküteri, Namet Dilli Salam 100 gr',
  'similarity': array([[0.86852831]])},
 {'productName': 'Şarküteri, Namet 7/24 Dana Macar Salam 60 gr',
  'similarity': array([[0.86510584]])},
 {'productName': 'Şarküteri, Polonez Macar Salam Fıstıklı 100 gr',
  'similarity': array([[0.86452803]])},
 {'productName': 'Şarküteri, Namet Dana Karabiberli-Fıstıklı Salam 100 gr',
  'similarity': array([[0.85931032]])},
 {'productName': 'Şarküteri, Polonez Macar Salam 110 g',
  'similarity': array([[0.85442593]])},
 {'productName': 'Şarküteri, Polonez Macar Salam 100 gr',
  'si

In [107]:
cart

['şarküteri namet fıstıklı macar salam', 'peynir muratbey burgu peyniri']